In [10]:
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.driver.driver_remote_connection import\
    DriverRemoteConnection
from gremlin_python.process.traversal import T
from gremlin_python.process.traversal import P
from gremlin_python.process.graph_traversal import __
import json
from gremlin_python.process.traversal import Order

In [11]:
server = '172.17.0.2'
port = 8182
endpoint = 'ws://' + server + ':' + str(port) + '/gremlin'
transport_args = {'max_content_length': 200000}
connection = DriverRemoteConnection(endpoint, 'g', **transport_args)
g = traversal().withRemote(connection)
# results = g.V('1','2','3').valueMap().toList()
# print(results)

In [13]:
g.addV('person').property('name', 'serg').as_('m'). \
        addV('person').property('name', 'alex').as_('v'). \
        addE('knows').from_('m').to('v').iterate()

[['addV', 'person'], ['property', 'name', 'serg'], ['as', 'm'], ['addV', 'person'], ['property', 'name', 'alex'], ['as', 'v'], ['addE', 'knows'], ['from', 'm'], ['to', 'v'], ['none'], ['values', '_ipython_canary_method_should_not_exist_'], ['values', '_ipython_canary_method_should_not_exist_']]

In [14]:
g.V().has('person', 'name', 'serg').valueMap().fold().next()

[{'name': ['serg']}]

In [15]:
    print(
        "marko: " + json.dumps(g.V().has('person', 'name', 'serg')
                              .valueMap().
                              next())
    )

marko: {"name": ["serg"]}


In [16]:
print(
        "who marko knows: " + json.dumps(g.V().has('person', 'name', 'serg')
                                        .out('knows')
                                        .valueMap()
                                        .next())
    )

who marko knows: {"name": ["alex"]}


In [17]:
# g.V().drop().iterate()
g.V().hasLabel('person').valueMap().next()

{'name': ['alex']}

## Load a dataset

In [18]:
g.V().drop().iterate()
g.io('/home/serg/projects/Bigdata/dataset/air-routes.graph.xml').read().iterate()

[['io', '/home/serg/projects/Bigdata/dataset/air-routes.graph.xml'], ['read'], ['none'], ['values', '_ipython_canary_method_should_not_exist_'], ['values', '_ipython_canary_method_should_not_exist_']]

## Explorara data

In [19]:
g.V().valueMap().next()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f9658ced3a0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f96435c3040>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f96435c3e50>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f96436f91f0>


{'code': ['SFO'],
 'type': ['airport'],
 'desc': ['San Francisco International Airport'],
 'country': ['US'],
 'longest': [11870],
 'city': ['San Francisco'],
 'elev': [13],
 'icao': ['KSFO'],
 'lon': [-122.375],
 'region': ['US-CA'],
 'runways': [4],
 'lat': [37.6189994812012]}

In [20]:
# Find uniq label
g.V().label().dedup().toList()

['airport', 'country', 'continent', 'version']

In [21]:
 # Count label
 g.V().groupCount().by(T.label).next()

{'continent': 7, 'country': 237, 'version': 1, 'airport': 3374}

In [22]:
 # Count label
 g.E().groupCount().by(T.label).next()

{'contains': 6748, 'route': 43400}

In [30]:
# Распределение степеней
g.V().groupCount().by(__.bothE().count()).next()

{0: 7,
 1: 75,
 2: 53,
 3: 11,
 4: 759,
 5: 56,
 6: 611,
 7: 42,
 8: 360,
 9: 17,
 10: 221,
 11: 16,
 12: 137,
 13: 19,
 14: 112,
 526: 1,
 15: 12,
 16: 87,
 17: 6,
 18: 77,
 19: 7,
 20: 68,
 21: 4,
 22: 58,
 23: 3,
 24: 37,
 25: 5,
 26: 43,
 27: 2,
 28: 37,
 29: 2,
 30: 20,
 542: 1,
 543: 1,
 31: 4,
 32: 29,
 33: 4,
 34: 24,
 546: 1,
 36: 16,
 37: 2,
 38: 18,
 39: 3,
 40: 24,
 41: 1,
 42: 25,
 44: 13,
 45: 2,
 46: 24,
 48: 14,
 49: 1,
 50: 9,
 51: 3,
 52: 7,
 54: 7,
 55: 2,
 56: 7,
 58: 7,
 59: 2,
 60: 6,
 62: 13,
 63: 1,
 64: 13,
 65: 1,
 66: 10,
 67: 2,
 579: 1,
 68: 15,
 70: 6,
 583: 1,
 72: 11,
 73: 3,
 74: 6,
 75: 1,
 76: 8,
 77: 2,
 78: 3,
 80: 10,
 81: 1,
 82: 6,
 84: 7,
 86: 8,
 87: 2,
 88: 5,
 90: 7,
 92: 5,
 94: 2,
 96: 8,
 98: 6,
 100: 1,
 102: 6,
 104: 3,
 106: 4,
 108: 3,
 110: 4,
 112: 6,
 113: 1,
 114: 3,
 115: 1,
 116: 2,
 118: 2,
 120: 9,
 122: 3,
 124: 6,
 126: 5,
 128: 7,
 130: 5,
 132: 2,
 134: 1,
 136: 6,
 137: 1,
 138: 3,
 140: 1,
 141: 1,
 142: 3,
 144: 1,
 146:

In [31]:
# Value in airport
g.V().hasLabel('airport').valueMap().next(1)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f96435d47c0>


[{'code': ['SFO'],
  'type': ['airport'],
  'desc': ['San Francisco International Airport'],
  'country': ['US'],
  'longest': [11870],
  'city': ['San Francisco'],
  'elev': [13],
  'icao': ['KSFO'],
  'lon': [-122.375],
  'region': ['US-CA'],
  'runways': [4],
  'lat': [37.6189994812012]}]

In [32]:
# Value in country
g.V().hasLabel('country').valueMap().next(1)

[{'code': ['AM'], 'type': ['country'], 'desc': ['Armenia']}]

In [33]:
# Value in continent
g.V().hasLabel('continent').valueMap().next(1)

[{'code': ['AS'], 'type': ['continent'], 'desc': ['Asia']}]

In [34]:
# Value in continent
g.V().hasLabel('version').valueMap().next(1)

[{'code': ['0.77'],
  'type': ['version'],
  'desc': ['Version: 0.77 Generated: 2017-10-06 16:24:40 UTC\nGraph created by Kelvin R. Lawrence\nPlease let me know of any errors you find in the graph.']}]

In [35]:
g.E().hasLabel('route').valueMap().next(10)

[{'dist': 1500},
 {'dist': 2010},
 {'dist': 2130},
 {'dist': 2450},
 {'dist': 2410},
 {'dist': 2577},
 {'dist': 1460},
 {'dist': 371},
 {'dist': 371},
 {'dist': 1630}]

In [36]:
g.E().hasLabel('contains').valueMap().next(10)

[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]

In [37]:
g.E().dedup().count().next()

50148

In [40]:
g.V().hasLabel('airport').groupCount().by('city').order().by(Order.asc).unfold().next(10)

[{'Udaipur': 1},
 {'Elim': 1},
 {'Indore': 1},
 {'San Jose': 4},
 {'Can Tho': 1},
 {'Chico': 1},
 {'Ishigaki': 1},
 {'International Falls': 1},
 {'Belo Horizonte': 2},
 {'Denizli': 1}]

In [98]:
g.V().hasLabel('airport').groupCount().by('runways').unfold().next(10)

[{1: 2316}, {2: 762}, {3: 225}, {4: 51}, {5: 14}, {6: 4}, {7: 1}, {8: 1}]

In [41]:
g.V().hasLabel('airport').groupCount().by('region').unfold().next(10)

[{'NZ-BOP': 3},
 {'MZ-A': 1},
 {'MZ-B': 1},
 {'IN-TN': 5},
 {'MZ-N': 1},
 {'PW-004': 1},
 {'MZ-I': 2},
 {'IN-TR': 1},
 {'BS-FP': 1},
 {'MZ-T': 1}]

In [113]:
g.V().hasLabel('airport').groupCount().by('runways').fold().next()

[{1: 2316, 2: 762, 3: 225, 4: 51, 5: 14, 6: 4, 7: 1, 8: 1}]

In [136]:
g.V().has('runways', P.eq(5)).values('desc').order().by(Order.asc).fold().next()

['Chicago Midway International Airport',
 'General Mitchell International Airport',
 'George Bush Intercontinental',
 'Gisborne Airport',
 'Hamilton International Airport',
 'Hartsfield - Jackson Atlanta International Airport',
 'Karup Airport',
 'Napier Airport',
 'Nelson Airport',
 'Paraparaumu Airport',
 'Shannon Airport',
 'Tauranga Airport',
 'Toronto Pearson International Airport',
 'Ufa International Airport']

In [16]:
g.E().valueMap().next()

{'dist': 973}

In [229]:
g.V().hasLabel('airport').outE('route').values('dist').order().by(Order.desc).mean().next() # limit(10).toList()

1164.7471658986176

In [244]:
random_id = g.V().hasLabel('airport').sample(1).id().next()
print(random_id)

225296


In [245]:
g.V(random_id).in_('route').valueMap().next()

{'code': ['AMS'],
 'type': ['airport'],
 'desc': ['Amsterdam Airport Schiphol'],
 'country': ['NL'],
 'longest': [12467],
 'city': ['Amsterdam'],
 'elev': [-11],
 'icao': ['EHAM'],
 'lon': [4.76388978958],
 'region': ['NL-NH'],
 'runways': [6],
 'lat': [52.3086013794]}

In [247]:
g.V(random_id).outE('route').outV().values('desc').next()

'Linköping City Airport'

In [261]:
 # Return the unique vertices of what's incoming to random_id and what's outgoing from random_id
 g.V(random_id).both().dedup().values('runway', 'city').fold().next()

['Amsterdam', 'Copenhagen']

In [272]:
 g.V(random_id).bothE().dedup().values('dist').order().by(Order.desc).fold().next()

[599, 599, 224, 224]

In [280]:
 # Return the unique vertices connected to either ends of edges that are connected to random_id (both incoming and outgoing)
 g.V(random_id).bothE().bothV().dedup().values('city').order().by(Order.desc).fold().next()

['Linköping', 'Copenhagen', 'Amsterdam']

In [290]:
g.V(random_id).in_('route').values('city').fold().next()

['Amsterdam', 'Copenhagen']

In [291]:
g.V(random_id).out('route').values('city').fold().next()

['Amsterdam', 'Copenhagen']

In [342]:
# Create subgraph
# g.V().hasLabel('airport').has('code', 'SYD').outE('route').subgraph('sg').inV().outE('route').subgraph('sg').cap('sg').next().traversal()
# g.E().hasLabel('route').subgraph('subGraph').cap('subGraph').next()
# g.V(random_id).emit().repeat(__.both().simplePath()).times(3).dedup().next()
# g.V().hasLabel('airport').has('code', 'SYD').emit().repeat(__.both().simplePath()).times(3).dedup().next()
sg = g.E().hasLabel('route').subgraph('subGraph').next()
print(sg)


e[{'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': '1nw8y-368-fth-36g'}}][4112-route->4120]


In [366]:
 subGraph = g.V().hasLabel('airport').has('code', 'SYD').outE('route').subgraph('x').cap('x').next()

{}


## has(), hasLabel(), hasId()

In [12]:
findId = g.V().hasLabel('airport').has('code', 'SYD').next()
g.V().hasLabel('airport').has('code', 'SYD').next() == g.V().has('airport', 'code', 'SYD').next() == g.V().hasId(findId).next()

True

In [99]:
 # Check data in Vertex by id
 print(findId)
 req = g.V().hasId(findId).valueMap().next()
 print(req)

v[9080968]
{'code': ['SYD'], 'type': ['airport'], 'desc': ['Sydney Kingsford Smith'], 'country': ['AU'], 'longest': [12999], 'city': ['Sydney'], 'elev': [21], 'icao': ['YSSY'], 'lon': [151.177001953125], 'region': ['AU-NSW'], 'runways': [3], 'lat': [-33.9460983276367]}


In [92]:
# Delete data by id
# req = g.V().hasId(findId).drop().iterate()
# print(req)

[['V'], ['hasId', v[7192816]], ['drop'], ['none']]


## Count

In [63]:
print(g.V().count().next())
print(g.V().hasLabel('country').count().next())
print(g.V().hasLabel('airport').count().next())

3618
237
3373


## valueMap()


In [100]:
g.V().has('airport', 'code', 'SYD').valueMap().next()

{'code': ['SYD'],
 'type': ['airport'],
 'desc': ['Sydney Kingsford Smith'],
 'country': ['AU'],
 'longest': [12999],
 'city': ['Sydney'],
 'elev': [21],
 'icao': ['YSSY'],
 'lon': [151.177001953125],
 'region': ['AU-NSW'],
 'runways': [3],
 'lat': [-33.9460983276367]}

In [11]:
# Check all data in request using next()
results = g.V(findId).values()
count = g.V(findId).values().count().next()
for c in range(count):
  print(results.next())

SYD
airport
Sydney Kingsford Smith
AU
12999
Sydney
21
YSSY
151.177001953125
AU-NSW
3
-33.9460983276367


In [12]:
req = g.V(findId).values('desc', 'runways')
print(f'{req.next()} airport has {req.next()} runways...')


Sydney Kingsford Smith airport has 3 runways...


## unfold() fold()

In [13]:

req = g.V(findId).valueMap().unfold()
count = g.V(findId).values().count().next()
for i in range(count):
  print(f'{str(i)}, {req.next()}')


0, {'code': ['SYD']}
1, {'type': ['airport']}
2, {'desc': ['Sydney Kingsford Smith']}
3, {'country': ['AU']}
4, {'longest': [12999]}
5, {'city': ['Sydney']}
6, {'elev': [21]}
7, {'icao': ['YSSY']}
8, {'lon': [151.177001953125]}
9, {'region': ['AU-NSW']}
10, {'runways': [3]}
11, {'lat': [-33.9460983276367]}


In [14]:
req = g.V(findId).valueMap().unfold().fold().next()
req

[{'code': ['SYD']},
 {'type': ['airport']},
 {'desc': ['Sydney Kingsford Smith']},
 {'country': ['AU']},
 {'longest': [12999]},
 {'city': ['Sydney']},
 {'elev': [21]},
 {'icao': ['YSSY']},
 {'lon': [151.177001953125]},
 {'region': ['AU-NSW']},
 {'runways': [3]},
 {'lat': [-33.9460983276367]}]

## hasNot(), not()

In [16]:
req = g.V().hasNot('region').valueMap()
count = g.V().hasNot('region').count().next()
for c in range(count):
    if c == 10:
        break
    print(req.next())

{'code': ['AL'], 'type': ['country'], 'desc': ['Albania']}
{'code': ['BB'], 'type': ['country'], 'desc': ['Barbados']}
{'code': ['BO'], 'type': ['country'], 'desc': ['Bolivia']}
{'code': ['BQ'], 'type': ['country'], 'desc': ['Bonaire - Sint Eustatius and Saba']}
{'code': ['BG'], 'type': ['country'], 'desc': ['Bulgaria']}
{'code': ['BI'], 'type': ['country'], 'desc': ['Burundi']}
{'code': ['KY'], 'type': ['country'], 'desc': ['Cayman Islands']}
{'code': ['CF'], 'type': ['country'], 'desc': ['Central African Republic']}
{'code': ['CC'], 'type': ['country'], 'desc': ['Cocos (Keeling) Islands']}
{'code': ['CD'], 'type': ['country'], 'desc': ['Democratic Republic of the Congo']}


In [17]:
same_count = g.V().not_(__.has('region')).count().next()

In [18]:
g.V().hasLabel('airport').next(3)

[v[5849112], v[5853208], v[5857304]]

In [19]:
g.E().hasLabel('route').count().next()

43400

In [20]:
results = g.V(findId).inE('route').outV().values('desc')
count = g.V(findId).inE('route').outV().count().next()
for c in range(count):
    print(results.next())

Melbourne International Airport
Wellington International Airport
Manila, Ninoy Aquino International Airport
Santiago, Comodoro Arturo Merino Benitez International Airport
Cairns International Airport
Darwin International Airport
Indira Gandhi International Airport
Auckland International Airport
Christchurch International Airport
Tokyo Haneda International Airport
Doha, Hamad International Airport
Alice Springs Airport
Dallas/Fort Worth International Airport
Soekarno-Hatta International Airport
Honolulu International Airport
Vancouver
Guangzhou Baiyun International Airport
Perth International Airport
Canberra International Airport
Seoul, Incheon International Airport
Sunshine Coast Airport
Dubai International Airport
Hong Kong - Chek Lap Kok International Airport
San Francisco International Airport
Singapore, Changi International Airport
Kuala Lumpur International Airport
Ho Chi Minh City, Tan Son Nhat International Airport
Ayers Rock Connellan Airport
Gold Coast Airport
Abu Dhabi Inter

## Get types of vertices

In [21]:
a = g.V().groupCount().by(T.label).next()
print(a)

{'continent': 7, 'country': 237, 'version': 1, 'airport': 3374}


In [22]:
g.V().hasLabel('airport').groupCount().by('country').select('FR','GR','BE').next()

{'FR': 58, 'GR': 39, 'BE': 5}

In [23]:
g.V().hasLabel('continent').group().\
    by('code').by(__.out().count()).next()

{'EU': 583, 'AS': 932, 'NA': 978, 'OC': 284, 'AF': 294, 'AN': 0, 'SA': 303}

In [24]:
g.V().hasLabel('airport').limit(5).group().\
    by('code').by('runways').next()

{'ORD': [8], 'BNA': [4], 'BWI': [3], 'IAD': [4], 'FLL': [2]}

In [4]:
g.V().has('region', 'AU-NSW').outE('route').fold().next()

value': {'relationId': 'goyrn-3yhi0-fth-3iiag'}}][6647832-route->5902360],
 e[{'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': 'g5gqs-3tnog-fth-1pge8'}}][6422560-route->2867264],
 e[{'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': 'g5h50-3tnog-fth-20oxs'}}][6422560-route->3391552],
 e[{'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': 'g5j44-3tu00-fth-1pge8'}}][6430752-route->2867264],
 e[{'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': 'g5jic-3tu00-fth-3iiag'}}][6430752-route->5902360],
 e[{'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': 'g5jwk-3tx5s-fth-1pge8'}}][6434848-route->2867264],
 e[{'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': 'g5kas-3tx5s-fth-1pjk0'}}][6434848-route->2871360],
 e[{'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': 'g5kp0-3tx5s-fth-1wtw8'}}][6434848-route->3211352],
 e[{'@type': 'janusgraph:RelationIdentifier', '@value': {'relati

In [9]:
g.V(findId).in_('route').count().next()

94

In [11]:
g.V().has('region', P.within('AU-NSW', 'AU-VIC')).\
    values('desc').fold().next()

['Albury Airport',
 'Moree Airport',
 'Coffs Harbour Airport',
 'Portland Airport',
 'Bathurst Airport',
 'Tamworth Airport',
 'Wagga Wagga City Airport',
 'Taree Airport',
 'Melbourne Essendon Airport',
 'Sydney Kingsford Smith',
 'Melbourne International Airport',
 'Parkes Airport',
 'Armidale Airport',
 'Broken Hill Airport',
 'Lord Howe Island Airport',
 'Merimbula Airport',
 'Narrandera Airport',
 'Gold Coast Airport',
 'Griffith Airport',
 'Sydney Bankstown Airport',
 'Avalon Airport',
 'Orange Airport',
 'Lismore Airport',
 'Moruya Airport',
 'Grafton Airport',
 'Narrabri Airport',
 'Warrnambool Airport',
 'Ballina Byron Gateway Airport',
 'Mildura Airport',
 'Port Macquarie Airport',
 'Dubbo City Regional Airport',
 'Newcastle Airport']

In [12]:
g.V().has('region', P.within('AU-NSW', 'AU-VIC')).\
    values('runways').mean().next()

1.875

In [17]:
 g.V(findId).as_('hk').out('route').in_('route').\
    where(P.neq('hk')).dedup().count().next()

1199

In [18]:
g.V(findId).out('route').tail(2).fold().next()

[v[6643736], v[6647832]]

In [22]:
g.V(findId).out().out().out().has('code', 'SYD').limit(10).path().by('desc').fold().next()

[path[Sydney Kingsford Smith, Melbourne International Airport, Wellington International Airport, Sydney Kingsford Smith],
 path[Sydney Kingsford Smith, Melbourne International Airport, Manila, Ninoy Aquino International Airport, Sydney Kingsford Smith],
 path[Sydney Kingsford Smith, Melbourne International Airport, Cairns International Airport, Sydney Kingsford Smith],
 path[Sydney Kingsford Smith, Melbourne International Airport, Darwin International Airport, Sydney Kingsford Smith],
 path[Sydney Kingsford Smith, Melbourne International Airport, Indira Gandhi International Airport, Sydney Kingsford Smith],
 path[Sydney Kingsford Smith, Melbourne International Airport, Auckland International Airport, Sydney Kingsford Smith],
 path[Sydney Kingsford Smith, Melbourne International Airport, Christchurch International Airport, Sydney Kingsford Smith],
 path[Sydney Kingsford Smith, Melbourne International Airport, Doha, Hamad International Airport, Sydney Kingsford Smith],
 path[Sydney Kings

In [25]:
 g.V(findId).as_('a').out().as_('b').out().as_('c').path().\
    by('code').from_('b').to('c').limit(10).fold().next()

[path[DFW, DCA],
 path[DFW, PBI],
 path[DFW, ANC],
 path[DFW, FLL],
 path[DFW, BWI],
 path[DFW, TPA],
 path[DFW, JFK],
 path[DFW, BNA],
 path[DFW, SAT],
 path[DFW, SEA]]

## Select

In [25]:
# Select the traversal of airports from HK that are 3 flights away
g.V(findId).as_('a').out().as_('b').out().as_('c').out().as_('d').\
    select('a', 'b', 'c', 'd').limit(10).fold().next()

[{'a': v[2867264], 'b': v[2871360], 'c': v[2867264], 'd': v[2871360]},
 {'a': v[2867264], 'b': v[2871360], 'c': v[2867264], 'd': v[2875456]},
 {'a': v[2867264], 'b': v[2871360], 'c': v[2867264], 'd': v[2891840]},
 {'a': v[2867264], 'b': v[2871360], 'c': v[2867264], 'd': v[2936896]},
 {'a': v[2867264], 'b': v[2871360], 'c': v[2867264], 'd': v[2945088]},
 {'a': v[2867264], 'b': v[2871360], 'c': v[2867264], 'd': v[2961472]},
 {'a': v[2867264], 'b': v[2871360], 'c': v[2867264], 'd': v[2994352]},
 {'a': v[2867264], 'b': v[2871360], 'c': v[2867264], 'd': v[3002544]},
 {'a': v[2867264], 'b': v[2871360], 'c': v[2867264], 'd': v[3010736]},
 {'a': v[2867264], 'b': v[2871360], 'c': v[2867264], 'd': v[3018928]}]

## Mean

In [27]:
g.V().hasLabel('airport').\
    local(__.out('route').count()).mean().next()

12.863070539419088

In [127]:
g.V().has('region','AU-NSW').order().by('code').values('code','city').limit(10).fold().next()

['ABX',
 'Albury',
 'ARM',
 'Armidale',
 'BHQ',
 'Broken Hill',
 'BHS',
 'Bathurst',
 'BNK',
 'Ballina']

## SuM()

In [126]:
g.V().hasLabel('airport').values('runways').sum().next()

4828

## Max()

In [7]:
g.V().hasLabel('airport').values('longest').max().next()

18045

# TASK, Select vertex to delete
Пример схемы с богами.
Написать метод рекурсивного обхода дерева с циклическими связями - и пометить узлы для удаления например


In [96]:
# Repeating traversal
'''
 * dedup() - Remove duplicates
 * where(neq("u861")) -  Exclude u861
 * repeat() - avoid combinatorial explosion!

 * emit() - modulator to “output” all the traversed vertices
 If emit() is placed after repeat(), it will “output” all vertices leaving the repeat-traversal. If emit() is placed before repeat(), it will “output” the vertices prior to entering the repeat-traversal.

 Controlling the recursion
 repeat(out("route").simplePath())) helps reducing the combinatory explosion due to cyclic loops but we still have a rapid graph expansion.
 '''

list_code = g.V()\
    .has('airport', 'code', 'SYD')\
    .as_("SYD")\
    .repeat(__.out("route")).times(2)\
    .emit()\
    .dedup()\
    .where(P.neq("SYD"))\
    .values("code")\
    .fold()\
    .next()
print(len(list_code))

0


In [107]:
id_for_delete = g.V()\
    .has('airport', 'code', 'SYD')\
    .as_("SYD")\
    .repeat(__.out("route")).times(2)\
    .emit()\
    .dedup()\
    .where(P.neq("SYD"))
counts =  g.V()\
    .has('airport', 'code', 'SYD')\
    .as_("SYD")\
    .repeat(__.out("route")).times(2)\
    .emit()\
    .dedup()\
    .where(P.neq("SYD"))\
    .count()\
    .next()
print('Total counts', counts)

for i in range(counts):
    if i > 10:
        break
    print(id_for_delete.next())



Total counts 0


In [106]:
# Delete data
g.V()\
    .has('airport', 'code', 'SYD')\
    .as_("SYD")\
    .repeat(__.out("route")).times(2)\
    .emit()\
    .where(P.neq("SYD"))\
    .drop()\
    .iterate()

[['V'], ['has', 'airport', 'code', 'SYD'], ['as', 'SYD'], ['repeat', [['out', 'route']]], ['times', 2], ['emit'], ['where', neq(SYD)], ['drop'], ['none'], ['values', '_ipython_canary_method_should_not_exist_'], ['values', '_ipython_canary_method_should_not_exist_']]

In [15]:
# g.E().has('airport', 'code', 'SYD').drop()
g.E().has('airport', 'code', 'SYD').fold().next()

[]

In [132]:
g.V(findId)\
    .repeat(__.out().simplePath())\
    .emit(__.has('code', 'TPE'))\
    .path()\
    .by('code')\
    .limit(50)\
    .fold()\
    .next()

[path[SYD, TPE],
 path[SYD, SIN, TPE],
 path[SYD, MEL, TPE],
 path[SYD, NRT, TPE],
 path[SYD, DXB, TPE],
 path[SYD, KUL, TPE],
 path[SYD, BKK, TPE],
 path[SYD, ICN, TPE],
 path[SYD, CAN, TPE],
 path[SYD, DPS, TPE],
 path[SYD, NKG, TPE],
 path[SYD, HGH, TPE],
 path[SYD, KMG, TPE],
 path[SYD, CSX, TPE],
 path[SYD, DEL, TPE],
 path[SYD, LAX, TPE],
 path[SYD, HKG, TPE],
 path[SYD, BNE, TPE],
 path[SYD, HNL, TPE],
 path[SYD, PEK, TPE],
 path[SYD, MNL, TPE],
 path[SYD, SGN, TPE],
 path[SYD, CTU, TPE],
 path[SYD, XIY, TPE],
 path[SYD, HAN, TPE],
 path[SYD, WUH, TPE],
 path[SYD, CKG, TPE],
 path[SYD, SFO, TPE],
 path[SYD, YVR, TPE],
 path[SYD, PVG, TPE],
 path[SYD, HND, TPE],
 path[SYD, CGK, TPE],
 path[SYD, SIN, CDG, TPE],
 path[SYD, SIN, MEL, TPE],
 path[SYD, SIN, NRT, TPE],
 path[SYD, SIN, FCO, TPE],
 path[SYD, SIN, DXB, TPE],
 path[SYD, SIN, KUL, TPE],
 path[SYD, SIN, BKK, TPE],
 path[SYD, SIN, ICN, TPE],
 path[SYD, SIN, PNH, TPE],
 path[SYD, SIN, FUK, TPE],
 path[SYD, SIN, CAN, TPE],
 pat

In [ ]:
'''
### Plan

[x] 1) I’ll share our previous demo with update graph functionality(we can get connection here, settings etc)
[x] 2) Sergey can create new notepad and connect to the graph using example from the demo
3) Work with Neptune Graph more:
    - prepare query for getting all elemnets of the subgraph(for example clamshell) and render them in the notepad
    - prepare query for getting all drop addresses for related clamshell
    - show 100 clamshells with connected house hold to the drop
    - mark/add to all elements of the subgraph(clamshell level) with the unique value property, render it and show that some elements have it
4) Lambda and Step Machine
    - step machine has a ist of regions(ATLANTA, FREEDOM)
    - create Lamdba which get all clamshell ids from the bucket(region)
    - get only 20 from that list of clamshell ids and run in parralel in the step function using lamdba function wich reognize all node names and calculate their count

Note:
1. Documentation for explorar data
Do we need drop verted before explorar data?

'''






# Pre working withe dataset

In [ ]:
'''
How do we know which Neptune cluster to access? The cell magics exposed by Neptune Notebooks use a configuration located by default under ~/graph_notebook_config.json At the time of initialization of the Sagemaker instance, this configuration is generated using environment variables derived from the cluster being connected to.
'''

%%bash

cat ~/graph_notebook_config.json

In [ ]:
%graph_notebook_config

# Reviewing the dataset

In [ ]:
# Check vertex of dataset
%%gremlin
g.V().groupCount().by(label).unfold()

In [ ]:
# Check edge of dataset
%%gremlin
g.E().groupCount().by(label).unfold()

In [ ]:
%%gremlin -p v,inv
g.V().has('airport','code','CZM').out('route').path().by('city')

In [ ]:
'''
Run the query below, select the Graph tab, select a node in the graph and click on the details button (the one with three .- one above the other). As you select different vertices the details view window will update. This is useful when you want to explore lots of properties for multiple vertices using the Graph view.
'''
%%gremlin
g.V().has('airport','code','CZM').
  out('route').
  path().
    by(valueMap('code','city','region','desc','lat','lon').
       order(local).
         by(keys))

In [ ]:
# Explorar data by ID of client
%%gremlin 

g.V('ed4982b00e323383583f30236e5b1f11').
    out('uses').     // traverse the users edge to get to IP addresses
    in('uses').      // go from the IP address vertex to other associated transient IDs
    in('has_identity').    //go from the found transient IDs to known users (persistent IDs)
    dedup().          // remove duplicate persistent IDs
    path()           // show a path from the unknown anonymous user to a known user

In [ ]:
%%gremlin

g.V('ed4982b00e323383583f30236e5b1f11').
    out('uses').     // traverse the users edge to get to IP addresses
    in('uses').      // go from the IP address vertex to other associated transient IDs
    in('has_identity').    //go from the found transient IDs to known users (persistent IDs)
    dedup().          // remove duplicate persistent IDs
    in('member').as('household').   //found the household associated with the found known user
    out('member').    // traverse back to other known users in this household
    out('has_identity').     //find other associated anonymous users associated to the known users
    out('visited','uses').     //look at the other websites/products browed by those anonymous users
    path().from('household')     //display the graph from household to associated clickstream events

In [ ]:
# Verify the data was inserted
%%gremlin
g.V().groupCount().by(label).unfold()


In [ ]:
# Verify the data was inserted
%%gremlin
g.E().groupCount().by(label).unfold()

In [ ]:
# Build a visualization of the graph 
%%gremlin -p v,ine,outv,oute,inv,oute,inv
g.V().hasLabel('League').inE().outV().outE().inV().outE().inV().path().by('name').by(label)

In [ ]:
# How many teams were in the league that season?
%%gremlin
g.V().hasLabel('Team').count()

In [ ]:
# How many teams were in the league that season?
%%gremlin
g.V('EPL-2019-20').in('CURRENT_LEAGUE').count()

In [ ]:
# Team info
%%gremlin
g.V().hasLabel('Team').valueMap()

In [ ]:
# Team and stadium info
%%gremlin
g.V().hasLabel('Team').
  project('name','stadium','city').
    by('name').
    by(out('STADIUM').values('name')).
    by(out('STADIUM').out('CITY').values('name'))

In [ ]:
# Teams based in London
%%gremlin
g.V().has('City','name','London').in('CITY').in('STADIUM').values('name')

In [ ]:
# Stadiums in London
%%gremlin -p v,outv
g.V().has('City','name','London').in('CITY').path().by('name')

In [ ]:
# Teams in London plus their stadiums
%%gremlin -p v,outv,outv
g.V().has('City','name','London').in('CITY').in('STADIUM').path().by('name')


In [ ]:
# Was Coventry in the league that season?
%%gremlin
g.V().has('Team','name','Coventry').
  fold().
  coalesce(unfold(),constant('Not in the EPL that year'))

In [ ]:
# Using text predicates to find sub-strings
%%gremlin
g.V().has('Team','name',containing('ou')).values('name')

In [ ]:
# Find everything the graph contains related to Arsenal

%%gremlin

g.V().hasId('Arsenal').
        project('names-and-age','stadium','city').
          by(valueMap().by(unfold())).
          by(out('STADIUM').values('name')).
          by(out('STADIUM').out('CITY').values('name')).unfold()

In [ ]:
# How many stadiums are in each city?
%%gremlin
g.V().hasLabel('Stadium').
  groupCount().
    by(out('CITY')).
  order(local).
    by(values,desc).
  unfold()

In [ ]:
# What year did each stadium open?
%%gremlin
g.V().hasLabel('Stadium').
  valueMap('name','opened').by(unfold()).
    order().
  by(select('opened'))

In [ ]:
# Stadiums ordered by descending capacity
%%gremlin
g.V().hasLabel('Stadium').
  order().
    by('capacity',desc).
  valueMap('name','capacity').
    by(unfold())

In [ ]:
# Number of teams founded in a given year

%%gremlin
g.V().hasLabel('Team').
  group().
    by('founded').
    by('name').
  order(local).
    by(keys).
  unfold()

In [ ]:
# In this case, the edge direction can be determined as there is an outE step included in the query.
g.V().hasLabel('airport').outE().inV().path().limit(5)

In [ ]:
'''
When no by modulators are provided, the visualizer will use the vertex and edge labels to annotate the elements of the diagram. However, when by modulators are used, it is not possible for the visualizer in all cases to decide on its own which path elements represent a vertex and which represent an edge. Here is an example of such a query:
'''

g.V().hasLabel('airport').outE().inV().path().by('code').by('dist').limit(5)

In [ ]:
# We can provide visualization hints for the query shown earlier as follows:

%%gremlin -p v,oute,inv
g.V().hasLabel('airport').outE().inV().path().by('code').by('dist').limit(5)

## Recomedation system

In [ ]:
%%gremlin

g.V().hasLabel('User').has('name', 'Terry').as_('user')  
  .bothE('FRIEND')    
    .has('strength', P.gt(1)).otherV()
    .aggregate('friends')
  .bothE('FRIEND')
    .has('strength', P.gt(1)).otherV()
    .where(P.neq('user')).where(P.without('friends'))
  .groupCount().by('name')
  .order(Scope.local).by(values, Order.decr)
  .next()


In [ ]:
# Lamdda
import json
import urllib.parse
import boto3

print('Loading function')

s3 = boto3.client('s3')


def lambda_handler(event, context):
    #print("Received event: " + json.dumps(event, indent=2))

    # Get the object from the event and show its content type
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = urllib.parse.unquote_plus(event['Records'][0]['s3']['object']['key'], encoding='utf-8')
    try:
        response = s3.get_object(Bucket=bucket, Key=key)
        print("CONTENT TYPE: " + response['ContentType'])
        return response['ContentType']
    except Exception as e:
        print(e)
        print('Error getting object {} from bucket {}. Make sure they exist and your bucket is in the same region as this function.'.format(key, bucket))
        raise e
              

In [ ]:
# - prepare query for getting all elemnets of the subgraph(for example clamshell) and render them in the notepad
%%gremlin
g.E().project('clamshell').
        by(id).
        by('clamshell').
        by(inV().id()).
        by(outV().id())
        
g.V(node_id).emit().repeat(both().simplePath()).dedup()

g.V().
  repeat(groupCount('m').
           by('name').
         outE().has(....).inV()).
    times(5).
  cap('m')

g.V().repeat(outE().order().by('clamshell').inV()).emit().times(3)

g.V().has('url','c94174b63350fd53').out('clamshell').dedup().path()
  

In [ ]:
# - prepare query for getting all drop addresses for related clamshell
%%gremlin 

g.V('id343434').out('clamshell').in('clamshell').dedup().toList()
g.V().has('name').out('clamshell').valueMap()      

%%gremlin -p v,oute,inv
g.V().has('airport','code','PSP').outE().inV().
  path().
    by(valueMap('code','city','region','desc','lat','lon').with(WithOptions.tokens)).
    by('dist').limit(10)

In [ ]:
#   - show 100 clamshells with connected house hold to the drop
g.V().hasLabel('clamshells').outE().inV().path().by('house_hold').limit(100)

g.V().has('country','NZ').out('route').has('country','AU').path().by('house hold').limit(100) #house hold dropn



In [ ]:
# - mark/add to all elements of the subgraph(clamshell level) with the unique value property, render it and show that some elements have it
g.V().hasLabel('person').properties('interesting').iterate()

# Test

In [79]:
g.V().count().next()

3619

In [80]:
g.E().count().next()

50148

In [101]:
g.V().next(3)

[v[10080264], v[10084360], v[10088456]]

In [139]:
g.V(10084360).valueMap().next()

{'code': ['ATL'],
 'type': ['airport'],
 'desc': ['Hartsfield - Jackson Atlanta International Airport'],
 'country': ['US'],
 'longest': [12390],
 'city': ['Atlanta'],
 'elev': [1026],
 'icao': ['KATL'],
 'lon': [-84.4281005859375],
 'region': ['US-GA'],
 'runways': [5],
 'lat': [33.6366996765137]}

In [143]:
g.E().next(3)

[e[{'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': 'lccn5-60554-fth-608aw'}}][10084360-route->10088456],
 e[{'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': 'lcfep-60554-fth-60bgo'}}][10084360-route->10092552],
 e[{'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': 'lcgzl-60554-fth-60emg'}}][10084360-route->10096648]]

In [240]:
g.V().hasLabel('airport').groupCount().by('country').next()

{'PR': 6,
 'PT': 14,
 'PW': 1,
 'PY': 2,
 'QA': 1,
 'AE': 10,
 'AF': 4,
 'AG': 1,
 'AI': 1,
 'AL': 1,
 'AM': 2,
 'AO': 14,
 'AR': 36,
 'AS': 1,
 'AT': 6,
 'RE': 2,
 'AU': 124,
 'AW': 1,
 'AZ': 5,
 'RO': 14,
 'BA': 4,
 'BB': 1,
 'RS': 2,
 'BD': 7,
 'RU': 120,
 'BE': 5,
 'BF': 2,
 'BG': 4,
 'RW': 2,
 'BH': 1,
 'BI': 1,
 'BJ': 1,
 'BL': 1,
 'BM': 1,
 'BN': 1,
 'BO': 15,
 'SA': 26,
 'SB': 17,
 'BQ': 3,
 'SC': 2,
 'BR': 115,
 'BS': 18,
 'SD': 5,
 'SE': 39,
 'BT': 1,
 'SG': 1,
 'BW': 4,
 'SH': 2,
 'SI': 1,
 'BY': 2,
 'BZ': 13,
 'SK': 2,
 'SL': 1,
 'SN': 3,
 'SO': 5,
 'CA': 203,
 'SR': 1,
 'SS': 1,
 'CC': 1,
 'ST': 1,
 'CD': 11,
 'CF': 1,
 'SV': 1,
 'CG': 3,
 'CH': 5,
 'SX': 1,
 'SY': 2,
 'CI': 1,
 'SZ': 1,
 'CK': 6,
 'CL': 17,
 'CM': 5,
 'CN': 209,
 'CO': 50,
 'CR': 13,
 'TC': 4,
 'TD': 1,
 'CU': 12,
 'TG': 1,
 'CV': 7,
 'TH': 32,
 'CW': 1,
 'CX': 1,
 'TJ': 4,
 'CY': 3,
 'CZ': 5,
 'TL': 1,
 'TM': 1,
 'TN': 8,
 'TO': 1,
 'TR': 48,
 'TT': 2,
 'DE': 33,
 'TV': 1,
 'TW': 9,
 'TZ': 8,
 'DJ': 1,
 

In [166]:
g.V().has('code','ORD').out().path().by('code').limit(10).fold().next()

[path[ORD, ATL],
 path[ORD, AUS],
 path[ORD, IAD],
 path[ORD, LGA],
 path[ORD, PHX],
 path[ORD, ELP],
 path[ORD, DUB],
 path[ORD, HKG],
 path[ORD, PVG],
 path[ORD, FCO]]

In [170]:
g.V().has('code','ORD').out().groupCount().by('country').next()

{'DE': 4,
 'PR': 1,
 'HK': 1,
 'BE': 1,
 'TW': 1,
 'JM': 1,
 'JO': 1,
 'JP': 2,
 'DK': 1,
 'DO': 1,
 'FR': 1,
 'QA': 1,
 'BR': 1,
 'BS': 1,
 'SE': 1,
 'UK': 2,
 'IE': 1,
 'US': 166,
 'CA': 13,
 'SV': 1,
 'AE': 2,
 'IN': 1,
 'CH': 1,
 'SX': 1,
 'KR': 1,
 'IS': 1,
 'CN': 2,
 'IT': 1,
 'MX': 6,
 'GT': 1,
 'KY': 1,
 'ES': 2,
 'TC': 1,
 'CR': 1,
 'PA': 1,
 'AT': 1,
 'VI': 1,
 'LC': 1,
 'PL': 2,
 'NL': 1,
 'TR': 1}

In [178]:
g.V().has('airport','code','DFW').values().fold().next()

['DFW',
 'airport',
 'Dallas/Fort Worth International Airport',
 'US',
 13401,
 'Dallas',
 607,
 'KDFW',
 -97.0380020141602,
 'US-TX',
 7,
 32.896800994873]

In [192]:
g.E().has('dist').count().next()

43400

In [196]:
g.E().next()

e[{'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': 'lccn5-60554-fth-608aw'}}][10084360-route->10088456]

In [229]:
g.E('10084360').valueMap().next()

ConnectionResetError: Cannot write to closing transport

In [183]:
g.V().has('airport','code','ORD').values('city').next()

'Chicago'

In [147]:
g.E().limit(1).valueMap().next()

{'dist': 811}

In [38]:
g.V().has('code').out('route').path().by('desc').next()

path[Hartsfield - Jackson Atlanta International Airport, Austin Bergstrom International Airport]

In [77]:
g.V().has('airport', 'code', 'SYD').valueMap().next()

{'code': ['SYD'],
 'type': ['airport'],
 'desc': ['Sydney Kingsford Smith'],
 'country': ['AU'],
 'longest': [12999],
 'city': ['Sydney'],
 'elev': [21],
 'icao': ['YSSY'],
 'lon': [151.177001953125],
 'region': ['AU-NSW'],
 'runways': [3],
 'lat': [-33.9460983276367]}

In [55]:
g.V().outE('route').count().next()

43400

In [56]:
g.V().hasLabel('airport').outE().count().next()

43400

In [63]:
g.V().outE('route').inV().values('desc').next()

'Austin Bergstrom International Airport'

In [74]:
g.V()\
    .has('airport', 'code', 'SYD')\
    .as_("SYD")\
    .repeat(__.out("route")).times(2)\
    .emit()\
    .dedup()\
    .where(P.neq("SYD")).count().next()

1203

In [234]:
g.E().limit().next()

ConnectionResetError: Cannot write to closing transport